In [1]:
import os
import math
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# Load fine-tuned LLaMA model (Replace with your model path)
tokenizer_path = "../Llama-3.2-3B"
ds_folder = "./dataset_txt_small/test"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.pad_token = tokenizer.eos_token

def compute_perplexity(checkpoint_path):
    model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
    files = os.listdir(ds_folder)
    BATCH_SIZE = 1
    N_FILES = len(files)

    device = "cuda"
    model.to(device)

    loss = 0
    losses = list()
    for i in tqdm(range((N_FILES-1)//BATCH_SIZE+1)):
        evaluation_texts = list()
        for filename in files[i*BATCH_SIZE:(i+1)*BATCH_SIZE]:
            with open(os.path.join(ds_folder, filename), 'r') as file:
                evaluation_texts.append(file.read())
        tokens = tokenizer(evaluation_texts, return_tensors="pt", truncation=False, padding=False, add_special_tokens=False)
        tokens = {k: v.to(device) for k, v in tokens.items()}
        # Compute loss
        with torch.no_grad():
            outputs = model(**tokens, labels=tokens["input_ids"])
            losses.append(outputs.loss.item())
            loss += outputs.loss.item() * len(evaluation_texts)

    # Compute perplexity
    perplexity = math.exp(loss/N_FILES)
    print(f"Perplexity: {perplexity:.2f}")

/scratch/ddp8196/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
compute_perplexity("./checkpoints/epoch_10")

100%|██████████| 83/83 [00:53<00:00,  1.56it/s]

Perplexity: 9.37


In [3]:
compute_perplexity("./dp_checkpoints/epoch_10")

100%|██████████| 83/83 [00:51<00:00,  1.60it/s]

Perplexity: 9.37
